In [ ]:
from mixer.models.modeling import MlpMixer, CONFIGS
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mixer.utils.data_utils import get_loader
from CKA_Google import cka, gram_rbf, gram_linear

In [ ]:
mixer = MlpMixer(CONFIGS['Mixer-B_16'])
mixer.load_from(np.load("mixer/checkpoint/Mixer-B_16.npz"))

In [15]:
mixer = MlpMixer(CONFIGS['Mixer-L_16'])
mixer.load_from(np.load("mixer/checkpoint/Mixer-L_16.npz"))

: 

: 

In [ ]:
class Args:
    pass

args = Args()
args.local_rank = -1
args.img_size = 224
args.train_batch_size = 16
args.eval_batch_size = 16

In [ ]:
train_loader, test_loader = get_loader(args)

In [ ]:
#https://discuss.pytorch.org/t/how-can-i-extract-intermediate-layer-output-from-loaded-cnn-model/77301/3
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

for i, layer in enumerate(mixer.layer):
    layer.register_forward_hook(get_activation(i))

# print(next(iter(train_loader)))
output = mixer(next(iter(train_loader))[0])
# activation

In [ ]:
def cka_rbf(a, b, sigma=1):
    return cka(
        gram_rbf(a.numpy(), sigma),
        gram_rbf(b.numpy(), sigma)
    )


def cka_linear(a, b, sigma=1):
    return cka(
        gram_linear(a.numpy(), sigma),
        gram_linear(b.numpy(), sigma)
    )


def compute_heatmap(mixer, activations, cka_func=cka_rbf):
    num_layers = len(mixer.layer)

    mean_activations = []
    for i in range(num_layers):
        mean_activation = activations[i].mean(axis=0)
        mean_activations.append(mean_activation)
    
    heatmap = np.zeros((num_layers, num_layers))
    for i in range(num_layers):
        for j in range(num_layers):
            heatmap[i][j] = cka_func(mean_activations[i], mean_activations[j])
    return heatmap


def display_heatmap(heatmap):
    sns.set()
    ax = sns.heatmap(heatmap, vmin=0, vmax=1)
    ax.invert_yaxis()
    plt.show()

In [ ]:
heatmap = compute_heatmap(mixer, activation)
display_heatmap(heatmap)